In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import P7_DataBreed
import p5_util

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from  sklearn import model_selection
import numpy as np

#-------------------------------------------------------------------------------
#
#-------------------------------------------------------------------------------
def arr_X_y_build(X,y):
    list_X = [np.array(x) for x in X]
    list_y = [Y for Y in y]

    weight  = list_X[0].shape[0]
    height  = list_X[0].shape[1]
    channel = list_X[0].shape[2]
    
    arr_X  = list_X[0].reshape((1, weight, height, channel))
    

    image_count = X.shape[0]
    list_image_error = list()
    for image_i in range(1,image_count) :
        try :
            arr_X\
            = np.append(arr_X,list_X[image_i].reshape((1, weight, height, channel)),axis=0)
        except ValueError :
            list_image_error.append(image_i)

    arr_y = np.array(y)
    return arr_X, arr_y
#-------------------------------------------------------------------------------

#-------------------------------------------------------------------------------
#
#-------------------------------------------------------------------------------
def arr_X_train_test_build(X,y,test_size=0.2):
    X_train, X_test, y_train,  y_test \
    = model_selection.train_test_split(X,y,test_size=test_size)

    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    print("Images for training...")
    arr_X_train, arr_y_train = arr_X_y_build(X_train, y_train)
    print("Images for testing...")
    arr_X_test, arr_y_test = arr_X_y_build(X_test, y_test)
    return arr_X_train, arr_X_test, arr_y_train, arr_y_test
#-------------------------------------------------------------------------------

In [ ]:
import p5_util
filename= './data/df_breed_image.dump'
df = p5_util.object_load(filename)
df.shape

**Execute following cell in order to update object version with no change in attributes**

* A backup instance of object is created. This backup instance is empty.
* If object instance is not defined then it is read from dmped file.
* object instance is copied back into backup instance
* Backup instance is deleted

**Building Data-model**

DataFrame with images and breed names are labels is built.

Labels are issued from encoding breed names. 

Labels are then used for training Keras network. 


**Building X_train, y_train, X_test, y_test**

In [ ]:
X=df['image']
y=df['label']
arr_X_train,arr_X_test, arr_y_train, arr_y_test = arr_X_train_test_build(X,y,test_size=0.6)


import p5_util
filename='./data/arr_X_y_train_test.dump'
p5_util.object_dump((arr_X_train,arr_X_test, arr_y_train, arr_y_test),filename)

In [ ]:
arr_X_train.shape,arr_X_test.shape,arr_y_train.shape,arr_y_test.shape

**Building Keras network**

In [ ]:
import p5_util
filename='./data/arr_X_y_train_test.dump'
(arr_X_train,arr_X_test, arr_y_train, arr_y_test) = p5_util.object_load(filename)

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense

my_VGG16 = Sequential()  # Création d'un réseau de neurones vide 

# Ajout de la première couche de convolution, suivie d'une couche ReLU
w=arr_X_train.shape[1]
h=arr_X_train.shape[2]
c=arr_X_train.shape[3]
print(w,h,c)
my_VGG16.add(Conv2D(64, (3, 3), input_shape=(w, h, c), padding='same', activation='relu'))

Using TensorFlow backend.


200 200 3


In [ ]:
# Ajout de la deuxième couche de convolution, suivie  d'une couche ReLU
my_VGG16.add(Conv2D(64, (3, 3), padding='same', activation='relu'))


In [3]:
# Ajout de la première couche de pooling
my_VGG16.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))


In [4]:
arr_y_train.shape[0]

365

In [5]:
# Ajout de la dernière couche fully-connected qui permet de classifier
my_VGG16.add(Flatten())  # Conversion des matrices 3D en vecteur 1D

print(arr_y_train.shape[0])
my_VGG16.add(Dense(arr_y_train.shape[0], activation='softmax'))

365


In [6]:
from keras import optimizers
sgd = optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)

#my_VGG16.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
my_VGG16.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [7]:
#for X_train, y_train in zip(list_X_train, list_y_train):
print(arr_X_train.shape)
my_VGG16.fit(arr_X_train, arr_y_train)

(365, 200, 200, 3)
Epoch 1/1
365/365 [==============================] - 22s 61ms/step - loss: 16.1181 - acc: 0.0000e+00


In [ ]:
my_VGG16.get_config()

**Test for images classification**

In [8]:
arr_result = my_VGG16.predict(arr_X_test)[0]

In [9]:
filename= './data/result_vgg16.dump'
p5_util.object_dump(arr_result,filename)

In [12]:
arr_result

array([6.8363668e-38, 0.0000000e+00, 1.8494062e-15, 2.2768897e-32,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 5.0137078e-29,
       0.0000000e+00, 0.0000000e+00, 4.1629119e-33, 4.5139040e-26,
       0.0000000e+00, 1.8933776e-24, 0.0000000e+00, 7.2826582e-24,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 6.0128037e-17, 0.0000000e+00, 2.3106209e-24,
       0.0000000e+00, 0.0000000e+00, 5.4673112e-32, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       2.5498006e-32, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       2.3368496e-32, 9.5936284e-26, 0.0000000e+00, 9.8330610e-10,
       1.5572405e-26, 0.0000000e+00, 0.0000000e+00, 1.1371664e-24,
       0.0000000e+00, 2.2376522e-31, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 1.6830685e-30, 0.0000000e+00, 3.7930571e